In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
import folium
from rasterio.plot import reshape_as_image
import branca.colormap as cm
from rasterio.transform import from_origin
from shapely.geometry import Point

Reads earthquake data, creates a geodataframe, and projects it to UTM for spatial analysis.

In [2]:
# Earthquake CSV (must have lat, lon, magnitude)
eq = pd.read_excel(r"D:\Projects\GIS\earthquake-risk-mapping-in-Nepal\data\cearthquakes.xlsx")  # columns: lat, lon, mag
gdf = gpd.GeoDataFrame(eq, geometry=gpd.points_from_xy(eq.Longitude, eq.Latitude), crs="EPSG:4326")
gdf = gdf.to_crs(epsg=32645)  # UTM for Nepal (meters)


In [ ]:
gdf.head()

Defines spatial grid for hazard calculation.

In [13]:
# Define bounds and resolution
xmin, ymin, xmax, ymax = gdf.total_bounds
pixel_size = 1000# meters
n_cols = int((xmax - xmin) / pixel_size)
n_rows = int((ymax - ymin) / pixel_size)

# Create grid coordinates
x = np.linspace(xmin + pixel_size/2, xmax - pixel_size/2, n_cols)
y = np.linspace(ymax - pixel_size/2, ymin + pixel_size/2, n_rows)  # top to bottom
xx, yy = np.meshgrid(x, y)
hazard_grid = np.zeros_like(xx)


In [14]:
#smaller the decay value, the wider the spread of intensity and vice versa
decay = 2

for idx, row in gdf.iterrows():
    #  calculates Epicenter coordinates
    ex, ey = row.geometry.x, row.geometry.y
    mag = row['Magnitude']
    
    # Distance from epicenter to all grid cells
    dist = np.sqrt((xx - ex)**2 + (yy - ey)**2)
    
    # Add intensity contribution
    #Converts distance and magnitude into a hazard value
    hazard_grid += mag / (dist/1000 + 1)**decay  # dist in km

In [15]:
#Normalization of hazard values between 0 and 1
hazard_grid_norm = (hazard_grid - hazard_grid.min()) / (hazard_grid.max() - hazard_grid.min())


In [16]:
# Open population raster (WorldPop)
pop_raster = r"D:\Projects\GIS\earthquake-risk-mapping-in-Nepal\data\npl_pop_2025_CN_100m_R2025A_v1.tif"
with rasterio.open(pop_raster) as pop_src: #safeway to open and close file
    pop_data = pop_src.read(1) #reads actual pixel value from file, reads only 1st layer
    pop_data = np.where(pop_data < 0, 0, pop_data)
    pop_transform = pop_src.transform      #for georeferencing
    pop_crs = pop_src.crs

    # Simple resample to match hazard grid size (use nearest for simplicity)
    %pip install -q scikit-image
    from skimage.transform import resize
    pop_resampled = resize(pop_data, hazard_grid_norm.shape, order=0, preserve_range=True) 
    #Takes population data and resizes it to match the hazard grid shape
    



After above code runs I have two grids hazard_grid_norm and pop_resampled that are perfectly aligned.


In [ ]:
#Multiplies hazard and population to estimate exposed population per grid cell and sums it up.
exposure = hazard_grid_norm * pop_resampled
total_exposed = exposure.sum()
print(f"Estimated exposed population: {int(total_exposed)}")


In [31]:
transform = from_origin(xmin, ymax, pixel_size, pixel_size)
with rasterio.open(
    "hazard_intensity.tif", "w",           #open and writes tif file
    driver="GTiff",
    height=hazard_grid_norm.shape[0],
    width=hazard_grid_norm.shape[1],
    count=1,
    dtype='float32',
    crs='EPSG:32645',
    transform=transform
) as dst:
    dst.write(hazard_grid_norm.astype('float32'), 1)

with rasterio.open(
    "exposure.tif", "w",
    driver="GTiff",
    height=exposure.shape[0],
    width=exposure.shape[1],
    count=1,
    dtype='float32',
    crs='EPSG:32645',
    transform=transform
) as dst:
    dst.write(exposure.astype('float32'), 1)


In [32]:
from rasterstats import zonal_stats
import geopandas as gpd

# Load district shapefile
districts = gpd.read_file(r"D:\Projects\GIS\earthquake-risk-mapping-in-Nepal\data\Nepal Districts Shapefile Download\03_DISTRICT\DISTRICT.shp")
districts = districts.to_crs("EPSG:32645")  # Match raster CRS

# Hazard stats (per district)
haz_stats = zonal_stats(districts, "hazard_intensity.tif", stats=["mean", "max"], geojson_out=True)

# Exposure stats (per district)
exp_stats = zonal_stats(districts, "exposure.tif", stats=["sum"], geojson_out=True)

# Attach stats to GeoDataFrame
districts["hazard_mean"] = [f["properties"]["mean"] for f in haz_stats]
districts["hazard_max"] = [f["properties"]["max"] for f in haz_stats]
districts["exposed_pop"] = [f["properties"]["sum"] for f in exp_stats]

# Simple Risk Index
districts["risk_index"] = districts["hazard_mean"] * districts["exposed_pop"]


In [33]:
districts

In [45]:


# Calculate population sum per district
pop_stats = zonal_stats(
    districts, 
    r"D:\Projects\GIS\earthquake-risk-mapping-in-Nepal\data\npl_pop_2025_CN_100m_R2025A_v1.tif",
    stats=["sum"], 
    geojson_out=True
)

# Add population to GeoDataFrame
districts["population"] = [f["properties"]["sum"] for f in pop_stats]


In [46]:
# Exposed population = population * normalized hazard (0-1)
districts["hazard_norm"] = districts["hazard_mean"] / districts["hazard_mean"].max()

# Total exposed population
total_exposed = districts["exposed_pop"].sum()
print(f"Estimated exposed population: {int(total_exposed)}")


In [39]:
districts["risk_index"] = districts["hazard_mean"] * districts["exposed_pop"]


In [41]:
# Normalize risk index for visualization (0-1)
districts["risk_norm"] = (districts["risk_index"] - districts["risk_index"].min()) / (districts["risk_index"].max() - districts["risk_index"].min())

In [42]:
# Ensure the districts GeoDataFrame has a CRS first (UTM 32645)
districts = districts.set_crs(epsg=32645, allow_override=True)

# Convert to latitude/longitude
districts = districts.to_crs(epsg=4326)

# Now the geometries are in lat/lon
print(districts.geometry.head())
print(districts.geometry.total_bounds)  # should be roughly [80,26,88,30] for Nepal


In [ ]:



nepal_bounds = districts.total_bounds
print(nepal_bounds)


#  Make sure risk_norm exists and is 0-1
districts["risk_norm"] = (districts["risk_index"] - districts["risk_index"].min()) / \
                         (districts["risk_index"].max() - districts["risk_index"].min())
districts["risk_norm"] = districts["risk_norm"].fillna(0).clip(0,1)


#  Create Folium map
m = folium.Map(location=[28.2, 84.0], zoom_start=7, tiles="CartoDB positron")
m.fit_bounds([[nepal_bounds[1], nepal_bounds[0]], [nepal_bounds[3], nepal_bounds[2]]])

colors= ["#f0f0f0", "#ffffb2", "#fecc5c", "#fd8d3c", "#e31a1c", "#800026"]
colormap = cm.LinearColormap(colors=colors, vmin=0, vmax=1, caption="Earthquake Risk Index")

folium.GeoJson(
    districts,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["risk_norm"]),
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.8
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["DISTRICT", "population", "hazard_mean", "exposed_pop", "risk_index"],
        aliases=["District:", "Population:", "Mean Hazard:", "Exposed Pop:", "Risk Index:"],
        localize=True,
        sticky=True,
        labels=True
    )
).add_to(m)

colormap.add_to(m)
m.save("earthquake_risk_map.html")
print("✅ Map saved. Nepal districts should now appear.")


